In [1]:
from collections import Counter
from datetime import datetime
import numpy as np
import os
import pandas as pd
import pytz
import re
import time
import tweepy

In [2]:

   
import requests
import json
from twython import Twython
import requests

In [3]:
api_key = "UN0Eh2pwcP9SBVHAm3TvNbKa9"
api_secrets = "nXDRsKkVXdDMUPz0UqtBHhhOhsXbNxeGlVZYolt9NrTydz5QBs"
access_token = '1529624890885668866-KnXbIjP9qGm0j4B5VhH1gVjdPzdAYn'
access_secret = '7wQqI55hoJv6lqr9oVow7F2YTTVJrAvvFrq2j2UKL3KTv'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAD4XdAEAAAAAiAAOuKsy9EBIxIkZua4n01%2FoN8U%3DTBzSO0BEpZk0sP5TxokVObpWCycFGNDVrccwIxAcCUxFB0vXmZ'
consumer_key = "dEZwUEs1bkh1cGoxOVN0dHpNeUo6MTpjaQ"
consumer_secret = "J-QfFYdLDrJNcZiU1KxFXbVS5BAAE_zr7Sl1F56oxniF42m2Xy"
client_id = ""
client_secret = ""


In [4]:
wordle_start = pytz.timezone("US/Pacific").localize(datetime(2021, 6, 19))
now = pytz.utc.localize(datetime.now()).astimezone(pytz.timezone("US/Pacific"))

wordle_id = (now-wordle_start).days
print("Today's wordle_id is: %d (%s)" % (wordle_id, now.strftime("%Y-%m-%d %H:%M PT")))



Today's wordle_id is: 353 (2022-06-07 01:57 PT)


In [5]:
search_url = "https://api.twitter.com/2/tweets/search/recent"

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields

###Parameters to pass in the twitter query
def create_query_params(next_token): 
     return{'query': "wordle %d" % wordle_id,'tweet.fields': 'author_id,created_at',  'max_results': 100, "next_token" : next_token}

#####
####
###Fields to get from twitter
###wordle_id, tweet.id, tweet.created_at, tweet.author.screen_name, tweet.text
###How many tweets - 5000
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    #print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

query_params = create_query_params(None);
json_response = connect_to_endpoint(search_url, query_params)
print(json.dumps(json_response, indent=4, sort_keys=True))



{
    "data": [
        {
            "author_id": "14467116",
            "created_at": "2022-06-07T13:57:10.000Z",
            "id": "1534172619892662272",
            "text": "Wordle 353 3/6\n\n\u2b1b\u2b1b\u2b1b\u2b1b\u2b1b\n\ud83d\udfe9\ud83d\udfe8\u2b1b\u2b1b\ud83d\udfe9\n\ud83d\udfe9\ud83d\udfe9\ud83d\udfe9\ud83d\udfe9\ud83d\udfe9"
        },
        {
            "author_id": "75208020",
            "created_at": "2022-06-07T13:57:10.000Z",
            "id": "1534172616600145920",
            "text": "Wordle 353 3/6\n\n\u2b1b\u2b1b\u2b1b\u2b1b\u2b1b\n\u2b1b\ud83d\udfe8\ud83d\udfe8\u2b1b\u2b1b\n\ud83d\udfe9\ud83d\udfe9\ud83d\udfe9\ud83d\udfe9\ud83d\udfe9"
        },
        {
            "author_id": "1520593678712115200",
            "created_at": "2022-06-07T13:57:08.000Z",
            "id": "1534172609088208896",
            "text": "Wordle 353 4/6\n\n\ud83d\udfe9\ud83d\udfe9\u2b1c\u2b1c\u2b1c\n\ud83d\udfe9\ud83d\udfe9\ud83d\udfe9\u2b1c\u2b1c\n\ud83d\udfe9\ud83d\udfe9\ud83d\u

In [6]:
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser

In [7]:
wordle_tweets = []

expected_tweets = 5000
count = 0 # Counting tweets per time period
max_count = 100 # Max tweets per time period
total_tweets = 0
flag = True
next_token = None
while total_tweets < expected_tweets:
    print("-------------------")
    print("Token: ", next_token)
    query_params = create_query_params(next_token);
    json_response = connect_to_endpoint(search_url, query_params)
    result_count = json_response['meta']['result_count']

    if 'next_token' in json_response['meta']:
        # Save the token to use for next call
        next_token = json_response['meta']['next_token']
        print("Next Token: ", next_token)
    if result_count is not None and result_count > 0 and next_token is not None:
        for tweet in json_response['data']:
            created_at = dateutil.parser.parse(tweet['created_at'])
            wordle_tweets.append((wordle_id, tweet["id"], created_at, tweet["author_id"], tweet["text"]))

        count += result_count
        total_tweets += result_count
        print("Total # of Tweets added: ", total_tweets)
        print("-------------------")
    
#print("Total number of results: ", total_tweets)
    
 #   wordle_tweets.append((wordle_id, tweet.id, tweet.created_at, tweet.author.screen_name, tweet.text))

print("Pulled %d tweets for wordle %d" % (len(wordle_tweets), wordle_id))

-------------------
Token:  None
Next Token:  b26v89c19zqg8o3fpyzm8yyrp4rvipkzfghgthktd6xkt
Total # of Tweets added:  100
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyzm8yyrp4rvipkzfghgthktd6xkt
Next Token:  b26v89c19zqg8o3fpyzm8yyrnn7yj7ah8myujvxqcpjst
Total # of Tweets added:  200
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyzm8yyrnn7yj7ah8myujvxqcpjst
Next Token:  b26v89c19zqg8o3fpyzm8yyrnmd0vgf79hcpkqv9emqnx
Total # of Tweets added:  300
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyzm8yyrnmd0vgf79hcpkqv9emqnx
Next Token:  b26v89c19zqg8o3fpyzm8yyrm4t62s1t4t5vtlv6tdyf1
Total # of Tweets added:  400
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyzm8yyrm4t62s1t4t5vtlv6tdyf1
Next Token:  b26v89c19zqg8o3fpyzm8yyrm3qsj0dkxom0ykqt8z2il
Total # of Tweets added:  500
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyzm8yyrm3qsj0dkxom0ykqt8z2il
Next Token:  b26v89c19zqg8o3fpyzm8yyh7ctrfgb7vxxsu7y

Next Token:  b26v89c19zqg8o3fpyzm8yy649tcsf70cwf9y2mehdkzh
Total # of Tweets added:  4597
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyzm8yy649tcsf70cwf9y2mehdkzh
Next Token:  b26v89c19zqg8o3fpyzm8yy62sogzzplv21vhmy7cltvh
Total # of Tweets added:  4697
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyzm8yy62sogzzplv21vhmy7cltvh
Next Token:  b26v89c19zqg8o3fpyzm8yy62rm9wmpcpstphcbjod0n1
Total # of Tweets added:  4797
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyzm8yy62rm9wmpcpstphcbjod0n1
Next Token:  b26v89c19zqg8o3fpyzm8yy61a9y8mzqlc98qtkmp42v1
Total # of Tweets added:  4897
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyzm8yy61a9y8mzqlc98qtkmp42v1
Next Token:  b26v89c19zqg8o3fpyzm8yy619f3tuoj9kyzeqsimoozh
Total # of Tweets added:  4997
-------------------
-------------------
Token:  b26v89c19zqg8o3fpyzm8yy619f3tuoj9kyzeqsimoozh
Next Token:  b26v89c19zqg8o3fpyzm8yxvmipugpdbu22rgxamn3vct
Total # of Tweets add

In [8]:
def is_valid_wordle_tweet(tweet, wordle_id): 
    text = (tweet.replace("Y", "y").replace("🟩", "Y")
                 .replace("M", "m").replace("🟨", "M")
                 .replace("N", "n").replace("⬛", "N").replace("⬜", "N"))
    
    for i in range(wordle_id-20, wordle_id+20):
        if i==wordle_id:
            continue
        if str(i) in text:
            return False
    
    if len(re.findall("Wordle %d" % wordle_id, text)) != 1:
        return False

    if re.match("Wordle %d [2-6]/6\n\n[YMN]{5}\n" % wordle_id, text) is None:
        return False
    return True

In [9]:
new_tweets_df = pd.DataFrame([tweet for tweet in wordle_tweets if is_valid_wordle_tweet(tweet[4], tweet[0])],
                             columns=["wordle_id", "tweet_id", "tweet_date", "tweet_username", "tweet_text"])
new_tweets_df

wordle_id             tweet_id                tweet_date  \
0           353  1534172624351158272 2022-06-07 13:57:11+00:00   
1           353  1534172619892662272 2022-06-07 13:57:10+00:00   
2           353  1534172616600145920 2022-06-07 13:57:10+00:00   
3           353  1534172609088208896 2022-06-07 13:57:08+00:00   
4           353  1534172597897900039 2022-06-07 13:57:05+00:00   
...         ...                  ...                       ...   
3720        353  1534150013001125893 2022-06-07 12:27:20+00:00   
3721        353  1534150010761355265 2022-06-07 12:27:20+00:00   
3722        353  1534150009477816326 2022-06-07 12:27:20+00:00   
3723        353  1534150000690724866 2022-06-07 12:27:17+00:00   
3724        353  1534149980516241408 2022-06-07 12:27:13+00:00   

           tweet_username                                         tweet_text  
0               264376328              Wordle 353 3/6\n\n🟩⬜⬜⬜🟩\n🟩🟨⬜⬜🟩\n🟩🟩🟩🟩🟩  
1                14467116              Wordle 353 3/6\n\n⬛⬛⬛⬛⬛\n🟩🟨⬛⬛🟩\n🟩🟩🟩🟩🟩  
2                75208020              Wordle 353 3/6\n\n⬛⬛⬛⬛⬛\n⬛🟨🟨⬛⬛\n🟩🟩🟩🟩🟩  
3     1520593678712115200       Wordle 353 4/6\n\n🟩🟩⬜⬜⬜\n🟩🟩🟩⬜⬜\n🟩🟩🟩⬜⬜\n🟩🟩🟩🟩🟩  
4               498500844  Wordle 353 5/6\n\n🟨🟨⬜⬜⬜\n⬜⬜🟨⬜🟨\n⬜🟩🟩⬜⬜\n⬜🟩🟩🟩⬜\n...  
...                   ...                                                ...  
3720             20447190       Wordle 353 4/6\n\n⬜⬜⬜🟨⬜\n⬜🟩🟩⬜🟩\n⬜🟩🟩⬜🟩\n🟩🟩🟩🟩🟩  
3721                61743  Wordle 353 5/6\n\n⬜⬜⬜⬜⬜\n⬜🟩⬜⬜⬜\n⬜🟩🟩⬜⬜\n🟩🟩🟩⬜⬜\n...  
3722            147991110  Wordle 353 6/6\n\n⬜🟨⬜⬜⬜\n⬜🟨⬜⬜⬜\n🟨⬜🟨⬜⬜\n🟨⬜🟩🟩🟨\n...  
3723           2663850053       Wordle 353 4/6\n\n🟨⬛⬛⬛⬛\n⬛🟩🟩⬛🟩\n⬛🟩🟩⬛🟩\n🟩🟩🟩🟩🟩  
3724             97106492              Wordle 353 3/6\n\n⬜🟨⬜⬜⬜\n⬜🟩🟩⬜⬜\n🟩🟩🟩🟩🟩  

[3725 rows x 5 columns]

In [10]:
previous_df = pd.read_csv("tweets.csv")
previous_df

wordle_id             tweet_id                 tweet_date  \
0             210  1482553374591660037  2022-01-16 03:20:43+00:00   
1             210  1482553387937898499  2022-01-16 03:20:46+00:00   
2             210  1482553422276698113  2022-01-16 03:20:55+00:00   
3             210  1482553436910628866  2022-01-16 03:20:58+00:00   
4             210  1482553445726908420  2022-01-16 03:21:00+00:00   
...           ...                  ...                        ...   
585533        338  1528914119616778240  2022-05-24 01:41:46+00:00   
585534        338  1528914119797129218  2022-05-24 01:41:46+00:00   
585535        338  1528914135207055360  2022-05-24 01:41:50+00:00   
585536        338  1528914145474641920  2022-05-24 01:41:52+00:00   
585537        338  1528914159466840067  2022-05-24 01:41:56+00:00   

       tweet_username                                         tweet_text  
0            bpszebes       Wordle 210 4/6\n\n⬛⬛🟨🟨⬛\n🟩⬛⬛⬛⬛\n🟩🟩🟨⬛⬛\n🟩🟩🟩🟩🟩  
1          cruisecoup       Wordle 210 4/6\n\n⬜⬜⬜⬜⬜\n🟩🟩🟨🟨⬜\n🟩🟩⬜🟩🟨\n🟩🟩🟩🟩🟩  
2          DestroVega       Wordle 210 4/6\n\n⬜⬜⬜🟨⬜\n⬜🟩⬜🟨⬜\n⬜🟩⬜🟨⬜\n🟩🟩🟩🟩🟩  
3         brenmardash              Wordle 210 3/6\n\n⬜⬜🟨⬜⬜\n🟨🟨⬜⬜⬜\n🟩🟩🟩🟩🟩  
4         KatieHowse2              Wordle 210 3/6\n\n⬛⬛🟨⬛⬛\n🟩🟩🟩⬛⬛\n🟩🟩🟩🟩🟩  
...               ...                                                ...  
585533     breathom02  Wordle 338 5/6\n\n⬜⬜⬜⬜⬜\n⬜⬜⬜🟨⬜\n⬜🟨🟩🟨⬜\n⬜🟩🟩🟩🟩\n...  
585534       _Sheka_J  Wordle 338 5/6\n\n⬜⬜⬜⬜⬜\n⬜⬜⬜⬜⬜\n⬜🟨🟨⬜⬜\n🟩🟩🟨⬜⬜\n...  
585535     atwhelchel  Wordle 338 5/6\n\n🟨⬛⬛⬛🟩\n⬛🟩⬛⬛🟩\n⬛🟩⬛🟩🟩\n⬛🟩🟩🟩🟩\n...  
585536        GoffieY              Wordle 338 3/6\n\n⬛🟨⬛⬛🟩\n⬛🟨🟨🟨🟩\n🟩🟩🟩🟩🟩  
585537  Allisa_wordle  Wordle 338 5/6\n\n⬛🟨⬛⬛⬛\n🟨🟨⬛⬛⬛\n⬛🟩🟩🟨⬛\n⬛🟩🟩⬛🟩\n...  

[585538 rows x 5 columns]

In [11]:
df = (pd.concat([previous_df,new_tweets_df])
      .drop_duplicates(subset="tweet_id")
      .sort_values(["wordle_id", "tweet_id"])
      .reset_index(drop=True))
df

wordle_id             tweet_id                 tweet_date  \
0             210  1482553374591660037  2022-01-16 03:20:43+00:00   
1             210  1482553387937898499  2022-01-16 03:20:46+00:00   
2             210  1482553422276698113  2022-01-16 03:20:55+00:00   
3             210  1482553436910628866  2022-01-16 03:20:58+00:00   
4             210  1482553445726908420  2022-01-16 03:21:00+00:00   
...           ...                  ...                        ...   
589258        353  1534172597897900039  2022-06-07 13:57:05+00:00   
589259        353  1534172609088208896  2022-06-07 13:57:08+00:00   
589260        353  1534172616600145920  2022-06-07 13:57:10+00:00   
589261        353  1534172619892662272  2022-06-07 13:57:10+00:00   
589262        353  1534172624351158272  2022-06-07 13:57:11+00:00   

             tweet_username                                         tweet_text  
0                  bpszebes       Wordle 210 4/6\n\n⬛⬛🟨🟨⬛\n🟩⬛⬛⬛⬛\n🟩🟩🟨⬛⬛\n🟩🟩🟩🟩🟩  
1                cruisecoup       Wordle 210 4/6\n\n⬜⬜⬜⬜⬜\n🟩🟩🟨🟨⬜\n🟩🟩⬜🟩🟨\n🟩🟩🟩🟩🟩  
2                DestroVega       Wordle 210 4/6\n\n⬜⬜⬜🟨⬜\n⬜🟩⬜🟨⬜\n⬜🟩⬜🟨⬜\n🟩🟩🟩🟩🟩  
3               brenmardash              Wordle 210 3/6\n\n⬜⬜🟨⬜⬜\n🟨🟨⬜⬜⬜\n🟩🟩🟩🟩🟩  
4               KatieHowse2              Wordle 210 3/6\n\n⬛⬛🟨⬛⬛\n🟩🟩🟩⬛⬛\n🟩🟩🟩🟩🟩  
...                     ...                                                ...  
589258            498500844  Wordle 353 5/6\n\n🟨🟨⬜⬜⬜\n⬜⬜🟨⬜🟨\n⬜🟩🟩⬜⬜\n⬜🟩🟩🟩⬜\n...  
589259  1520593678712115200       Wordle 353 4/6\n\n🟩🟩⬜⬜⬜\n🟩🟩🟩⬜⬜\n🟩🟩🟩⬜⬜\n🟩🟩🟩🟩🟩  
589260             75208020              Wordle 353 3/6\n\n⬛⬛⬛⬛⬛\n⬛🟨🟨⬛⬛\n🟩🟩🟩🟩🟩  
589261             14467116              Wordle 353 3/6\n\n⬛⬛⬛⬛⬛\n🟩🟨⬛⬛🟩\n🟩🟩🟩🟩🟩  
589262            264376328              Wordle 353 3/6\n\n🟩⬜⬜⬜🟩\n🟩🟨⬜⬜🟩\n🟩🟩🟩🟩🟩  

[589263 rows x 5 columns]

In [12]:
df.groupby("wordle_id")["wordle_id"].count()

wordle_id
210    2837
211    2894
212    2399
213    2119
214    2703
       ... 
335    3311
336    3635
337    3666
338    3651
353    3725
Name: wordle_id, Length: 128, dtype: int64

In [13]:
if not os.path.exists("updated"):
    os.mkdir("updated")

df.to_csv("updated/tweets.csv", index=False)

In [14]:
!kaggle datasets metadata -p updated benhamner/wordle-tweets

Downloaded metadata to updated/dataset-metadata.json


In [15]:
!kaggle datasets version -m "added tweets" -p updated

Starting upload for file tweets.csv
100%|██████████████████████████████████████| 92.5M/92.5M [00:37<00:00, 2.56MB/s]
Upload successful: tweets.csv (92MB)
404 - Not Found
